In [35]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from PIL import Image
import html
from markdownify import markdownify 

#endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"
endpoint_url = "https://query.wikidata.org/sparql"

query_txt = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?textItem ?kuratorLabel ?textUrl
WHERE
{
  <placeholder>
  ?textItem cpsp:P46 ?kuratorStatement. 
  ?kuratorStatement cpsps:P46 ?kuratorItem. 
  ?kuratorItem rdfs:label ?kuratorLabel.
  ?textItem cpsp:P57 ?urlstatement. 
  ?urlstatement cpsps:P57 ?textUrl. 
}"""

query_img ="""
SELECT DISTINCT ?artwork ?artworkLabel ?image ?width ?height WHERE {
  ?artwork wdt:P31 wd:Q3305213;
    wdt:P170 wd:Q41406;
    wdt:P18 ?image;
    wdt:P2048 ?height;
    wdt:P2049 ?width.
  FILTER(?width > ?height)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 25
"""

query_graph = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?x ?y
WHERE
{
  ?a cpsp:P2 ?c.
  ?c cpsps:P2 ?d.
  ?a rdfs:label ?x.
  ?d rdfs:label ?y.

}LIMIT 1"""

query_graph2 = """
SELECT ?a ?b ?c
WHERE
{
  ?a rdfs:label ?c
}LIMIT 100"""


def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

"""def get_text(textitem_id):
    q = ""
    if textitem_id:
        q = query_txt.replace("<placeholder>", "cps:"+textitem_id+" cpsp:P46 ?kuratorStatement.")
    else:
        q = query_txt.replace("<placeholder>","")
    results_txt = run_query(endpoint_url, q)
    for item in results_txt["results"]["bindings"]:
        print('Wikibase link: ' + '[' + item['textItem']['value'] + ']' + '(' + item['textItem']['value'] + ')' + '\n')
        print('Kurator: ' + item['kuratorLabel']['value'] + '\n')
        headers = {'User-Agent': 'Ex_Books_conference_bot/0.0 (https://github.com/SimonXIX/Experimental_Books_workshop; ad7588@coventry.ac.uk)'}
        r = requests.get(item['textUrl']['value'], headers=headers, stream=True)
        text = str(r.text) #changed from r.content
        text = text.removeprefix("<!DOCTYPE html>") #changed from "b'<!DOCTYPE html>"
        print(markdownify(text)) #changed
"""
def get_delay(date):
    try:
        date = datetime.datetime.strptime(date, '%a, %d %b %Y %H:%M:%S GMT')
        timeout = int((date - datetime.datetime.now()).total_seconds())
    except ValueError:
        timeout = int(date)
    return timeout

# def fetch_image_by_url(url, headers):
#     r = requests.get(url, headers=headers, stream=True)
#     if r.status_code == 200:
#         im = Image.open(r.raw)
#         return im
#     if r.status_code == 500:
#         return None
#     if r.status_code == 403:
#         return None
#     if r.status_code == 429:
#         timeout = get_delay(r.headers['retry-after'])
#         print('Timeout {} m {} s'.format(timeout // 60, timeout % 60))
#         time.sleep(timeout)
#         fetch_image_by_url(url, headers)

def get_img(partOfItem_id):
    print("")
    print("<link rel='stylesheet' href='./section-fullimg.css'>")

    q = ""
    print("<div class='navig'>")

    print("<input type='button' class='btn btn-secondary arrow arrowleft' onclick='openPrev()' value='&#10092;'/>")

    #print("<div class='stop-scrolling'>")

    if partOfItem_id:
        q = query_img.replace("<placeholder>", "?partOfStatement cpsps:P6 cps:"+partOfItem_id+".")
    else:
        q = query_img.replace("<placeholder>","")
    results_img = run_query(endpoint_url, q)


    if results_img["results"]["bindings"]:
      # Extract all image URLs into a list
      img_urls = [item["image"]["value"] for item in results_img["results"]["bindings"]]
      img_urls_js_array = ', '.join(f'"{url}"' for url in img_urls)  # Convert to JavaScript array

      # Output the first image and JavaScript logic
      print(f'<img class="img_full" src="{img_urls[0]}" alt="Image">')
      
    #print("</div>")

    print("<input type='button' class='btn btn-secondary arrow arrowright' onclick='openNext()' value='&#10093;'/>")
    print(f'''<script>
          // Store image URLs in a JavaScript array
          const imgUrls = [{img_urls_js_array}];
          let currentIndex = 0;

          function openNext() {{
              currentIndex = (currentIndex + 1) % imgUrls.length; // Increment index and loop back to 0
              const image = document.getElementsByClassName("img_full")[0];
              image.src = imgUrls[currentIndex]; // Update image source
          }}          
                    
          function openPrev() {{
            currentIndex = (currentIndex - 1 + imgUrls.length) % imgUrls.length; // Ensure index wraps around correctly
            const image = document.getElementsByClassName("img_full")[0];
            image.src = imgUrls[currentIndex]; // Update image source
          }}

      </script>''')

    print("</div>")

get_img("")

print('''
<table class="table table-hover">
  <thead>
    <tr>
      <th scope="col">Name</th>
      <th scope="col">Year</th>
      <th scope="col">Wikidata</th>
    </tr>
  </thead>
  <tbody>
    <tr class="table-active">
      <td>Column content</td>
      <td>Column contentcontentcontentcontentcontentcontentcontentcontentcontentcontentcontent content</td>
      <td>Column content</td>
    </tr>
  </tbody>
</table>''')



<link rel='stylesheet' href='./section-fullimg.css'>
<div class='navig'>
<input type='button' class='btn btn-secondary arrow arrowleft' onclick='openPrev()' value='&#10092;'/>
<img class="img_full" src="http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%20-%20Death%20at%20the%20helm.jpg" alt="Image">
<input type='button' class='btn btn-secondary arrow arrowright' onclick='openNext()' value='&#10093;'/>
<script>
          // Store image URLs in a JavaScript array
          const imgUrls = ["http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%20-%20Death%20at%20the%20helm.jpg", "http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%20-%20Death%20in%20the%20Sickroom%20-%20MM.M.00214%20-%20Munch%20Museum.jpg", "http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%2C%20D%C3%B8dskamp.JPG", "http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%20-%20Dance%20by%20the%20Sea%20-%20MM.M.00180%20-%20Munch%20Museum%20%28cropped%29.

In [36]:
#get_text("Q209")

In [37]:
#get_img("")

<hr>

In [38]:
#get_graph()